In [1]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from matplotlib import pyplot as plt
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score
import pandas as pd
import numpy as np
from sklearn import tree
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')
from sklearn.preprocessing import scale
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn import metrics
import functions as ff
from functions import *

In [2]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
X_train = df_train.drop(columns=['id', 'Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults'])
X_test = df_test.drop(columns=['id'])
Y_train = df_train[['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults']]

In [3]:
ff.transformacion(X_train)
ff.transformacion(X_test)

In [4]:
X_train_t, X_test_t, y_train_t, y_test_t = train_test_split(X_train, Y_train, test_size=0.2)

In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import BaggingClassifier

bagged_tree = BaggingClassifier(estimator=DecisionTreeClassifier(), n_estimators=20, max_samples=0.7, max_features=0.6)
bagged_tree.get_params()

{'bootstrap': True,
 'bootstrap_features': False,
 'estimator__ccp_alpha': 0.0,
 'estimator__class_weight': None,
 'estimator__criterion': 'gini',
 'estimator__max_depth': None,
 'estimator__max_features': None,
 'estimator__max_leaf_nodes': None,
 'estimator__min_impurity_decrease': 0.0,
 'estimator__min_samples_leaf': 1,
 'estimator__min_samples_split': 2,
 'estimator__min_weight_fraction_leaf': 0.0,
 'estimator__monotonic_cst': None,
 'estimator__random_state': None,
 'estimator__splitter': 'best',
 'estimator': DecisionTreeClassifier(),
 'max_features': 0.6,
 'max_samples': 0.7,
 'n_estimators': 20,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [6]:
params={'estimator__ccp_alpha':[0.0,0.1,0.2],
        'estimator__max_depth':[5,7,10],
        'estimator__max_features': [0.3,0.5,0.7],
        'estimator__min_samples_split': [4,6,8],
        'estimator__min_samples_leaf': [5, 10, 15],
        'n_estimators': [10, 15, 20]
        }

scoring = ['accuracy', 'roc_auc']

n_cv=4 

grid_solver = GridSearchCV(estimator = bagged_tree,
                   param_grid = params, 
                   scoring = scoring,
                   cv = n_cv,
                   refit = 'roc_auc',
                   verbose = 1)

model = MultiOutputClassifier(estimator=grid_solver)

In [7]:
model_result=model.fit(X_train_t,y_train_t)

Fitting 4 folds for each of 729 candidates, totalling 2916 fits
Fitting 4 folds for each of 729 candidates, totalling 2916 fits
Fitting 4 folds for each of 729 candidates, totalling 2916 fits
Fitting 4 folds for each of 729 candidates, totalling 2916 fits
Fitting 4 folds for each of 729 candidates, totalling 2916 fits
Fitting 4 folds for each of 729 candidates, totalling 2916 fits
Fitting 4 folds for each of 729 candidates, totalling 2916 fits


In [8]:
yhat_test_t=model_result.predict(X_test_t)
print('Roc Auc Tree:', roc_auc_score(y_true=y_test_t, y_score=yhat_test_t))
print('Accuracy:', accuracy_score(y_true=y_test_t, y_pred=yhat_test_t))

Roc Auc Tree: 0.659020602304995
Accuracy: 0.31815816857440166


In [9]:
yhat_train_t=model_result.predict(X_train_t)
print('Roc Auc Tree:', roc_auc_score(y_true=y_train_t, y_score=yhat_train_t))
print('Accuracy:', accuracy_score(y_true=y_train_t, y_pred=yhat_train_t))

Roc Auc Tree: 0.6678842785929291
Accuracy: 0.35095934959349595


In [10]:
y_pred_test=np.array(model_result.predict_proba(X_test))[:,:,1].T

In [11]:
importances=pd.DataFrame([X_test.columns,final_model.feature_importances_], index=["feature","importance"]).T
print(importances.sort_values("importance", ascending = False).head(10))

NameError: name 'final_model' is not defined

In [12]:
submission=pd.read_csv("sample_submission.csv")
submission.iloc[:,1:8]=y_pred_test
submission.head()

,id,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,19219,0.311676,0.007080,0.007277,0.000169,0.040598,0.184988,0.372768
1,19220,0.219209,0.034616,0.026060,0.000169,0.118144,0.205046,0.356097
2,19221,0.005090,0.046618,0.033485,0.000957,0.026301,0.285740,0.442349
3,19222,0.122114,0.071650,0.003520,0.016297,0.021006,0.435894,0.377119
4,19223,0.003955,0.008291,0.003887,0.000749,0.007441,0.580712,0.435556


In [13]:
submission.to_csv("8th_predict_submission.csv", index = False)